In [ ]:
from nornir import InitNornir
import pynetbox
from jinja2 import Environment, FileSystemLoader
import json

In [ ]:
# Init Nornir
nr = InitNornir(
    inventory={
        "plugin": "NetBoxInventory2",
        "options": {
            "nb_url": "http://10.30.1.226:8000",
            "nb_token": "dc5b766afda60e647a9fcc384c736a9689da8618",
            "group_file": "./inventory/groups.yml",
            "defaults_file": "./inventory/defaults.yml",
        },
    },
)

In [ ]:
# Init pynetbox
nb_url = nr.config.inventory.options['nb_url']
nb_token = nr.config.inventory.options['nb_token']
nb = pynetbox.api(
    nb_url,
    token=nb_token
)

In [ ]:
templates_path = "./templates/"

In [ ]:
level = 'error'
kind = ['emergency','alert','critical','error','warning','notification','informational','debugging',]
set_kind = tuple(kind)
if level in kind:
    log_level = set_kind.index(level)
    if log_level in range(2):
        netbox_level = 'danger'
    elif log_level in range(2,5):
        netbox_level = 'warning'
    elif log_level in range(5,7):
        netbox_level = 'info'
    else: netbox_level = 'success'

print(netbox_level)

In [ ]:
def template_fill(*args,**kwargs):
    template_file = "netbox_journals.template"
    environment = Environment(loader=FileSystemLoader(templates_path)) # загружаем шаблон для заполнения
    template = environment.get_template(template_file)
    content = None
    
    kind = ['emergency','alert','critical','error','warning','notification','informational','debugging',]
    set_kind = tuple(kind)
    if level in kind:
        log_level = set_kind.index(level)
        if log_level in range(2):
            netbox_level = 'danger'
            wow = '!!!'
        elif log_level in range(2,5):
            netbox_level = 'warning'
            wow = '!'
        elif log_level in range(5,7):
            netbox_level = 'info'
            wow = '.'
        else: 
            netbox_level = 'success'
            wow = ''
    print(netbox_level)
    
    try:
        content = template.render( # заполняем шаблон
                                assigned_object_type = 'dcim.device',
                                assigned_object_id = '12',
                                kind = netbox_level,
                                comments = comment + '{}'.format(wow)
                                )
    except: print("not all arguments have been transmitted...")
    else:
        print("Filling in the template...\n{}".format(content)) 
    return content 

#level = 'emergency'
#comment = 'All for testing'
comment,level = 'All for testing','emergency'
content = [json.loads(template_fill(level,comment))]
#nb.extras.journal_entries.create(content)

journals = list(nb.extras.journal_entries.all())
for journal in journals:
    print(journal.comments)